# This notebook iterates through the XML for data retrieval

In [1]:
import os
import numpy as np
import xml.etree.cElementTree as et
path = r'N:\AUG-ImageRepository\loc_img_db\21254\2012-08-10'

# below every part of XML containg ID's that should be removed will be accessed

In [5]:
def edit_exam_url(tree, path):
    root = tree.getroot()
    parent_map = dict((c, p) for p in tree.getiterator() for c in p)
    walkAll = tree.getiterator()
    for child in walkAll:
        if 'ExamURL' in str(child):
            Exam_URL_old = child.text
            #series type
            series_type = parent_map[parent_map[parent_map[child]]].findall('Type')[0].text
            #image type
            image_type = parent_map[parent_map[child]].findall('ImageType')[0][0].text
            #image number
            im_number = parent_map[parent_map[child]].findall('ImageNumber')[0].text
            #lateriality
            laterality = parent_map[parent_map[child]].findall('Laterality')[0].text
            #im_number
            im_number = parent_map[parent_map[child]].findall('ImageNumber')[0].text + ".tif"
            #aquasition time
            hour = parent_map[parent_map[child]].findall('AcquisitionTime')[0][0][0].text
            minute = parent_map[parent_map[child]].findall('AcquisitionTime')[0][0][1].text
            second = parent_map[parent_map[child]].findall('AcquisitionTime')[0][0][2].text
            date = hour +"-"+ minute +"-"+ str(int(float(second)))
            
            #edit the string format so it adds zeros when missing (math orig format)
            date_str = date.split('-')
            for string in date_str:
                if len(string) < 2:
                    index = date_str.index(string)
                    string = "0"+string
                    date_str[index] = string

            date = "-".join(date_str)
            
            #changing the slahes in path
            path = "/".join(path.split('\\'))
            Exam_URL = path+"/"+series_type+"/"+image_type+"/"+laterality+"/"+date+"/"+im_number
            #set the Exam URL to correct
            child.text = Exam_URL
    write_loc = os.path.join(path, 'anom_explore_corrected_url.xml')
    print(write_loc)
    tree.write(write_loc)
    return

def anonymize_xml(path):
    file_path = os.path.join(path, 'export.xml')
    tree = et.ElementTree()
    tree.parse(file_path)
    root = tree.getroot()
    parent_map = dict((c, p) for p in tree.getiterator() for c in p)

    #set birth day to first day of the month
    patient= root[0][2]
    for child in patient:
        #print(child)
        if 'Birthdate' in str(child):
            for children in child:
                #print(children[2])
                day = children[2]
                day.text = str(1)
                day.set('updated', 'yes')
    #Get ID node from Patient parent
    for child in root[0][2]:
        if 'ID' in str(child) and 'Patient' not in str(child):
            #print(child)
            parent_map[child].remove(child)

    #removal of Study ID
    for child in patient:
        if 'Study' in str(child):
            study = child
            #print(study)
            for children in study:
                #print(children)
                if 'ID' in str(children) and 'Study' not in str(children):
                   #print(children)
                    parent_map[children].remove(children)


    #removal of series ID
    for child in study:
        if 'Series' in str(child):
            #print(child)
            series = child
            for children in series:
                if 'ID' in str(children) and 'Series' not in str(children) and 'Prog' not in str(children):
                    #print(children)
                    parent_map[children].remove(children)

    #removal of series reference        
    for child in series:
        if 'ReferenceSeries' in str(child):
            #print(child)
            ReferenceSeries = child
            for children in ReferenceSeries:
                if 'ID' in str(children) and 'Series' not in str(children) and 'Prog' not in str(children):
                    #print(children)
                    parent_map[children].remove(children)

    for i in list(root.iter()):
        #remove all UID'S
        if 'PatientID' in str(i):
            #print(i)
            parent_map[i].remove(i)
        if 'LastName' in str(i):
            #print(i)
            parent_map[i].remove(i)
        if 'FirstNames' in str(i):
            #print(i)
            parent_map[i].remove(i)

        if 'UID' in str(i):
            #print(i)
            #print(parent_map[i])
            parent_map[i].remove(i)
            
    edit_exam_url(tree, path)
    return

# Below block iterates through the files and anomymizes the correct nodes

The below code corrects the url and writes the new .xml file as export_corrected_url

The code below removes personal inforamtion and corrects URL and writes the result specifices in function edit_exam_url

In [6]:
import os
path = r'N:\AUG-ImageRepository\anonymized_xml\loc_img_db'
folder = os.listdir(path)
k = 0
i = 0
for sub_folder in folder:
    
    sub_path = os.path.join(path, sub_folder)
    sub_dir = os.listdir(sub_path)
    
    for sub_files in sub_dir:
        #print(sub_files)
        scnd_sub_dir = os.path.join(path, sub_folder, sub_files)
        scnd_sub_files = os.listdir(scnd_sub_dir)
        for scnd_sub_files in scnd_sub_files:
            if scnd_sub_files.endswith('.xml'):
                file_path = scnd_sub_dir
                try:
                    anonymize_xml(file_path)
                    i += 1
                except:
                    k += 1
                    print("Number of corrupt files are{}".format(k))

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/24016/2010-11-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/24016/2010-11-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21460/2009-12-02\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21460/2009-12-02\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21254/2012-08-10\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21254/2012-08-10\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21254/2012-08-09\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21254/2012-08-09\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21254/2010-11-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/21254/2010-11-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2012-08-09\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2012-07-12\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2012-07-12\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2012-06-28\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2012-06-28\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2011-12-22\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2011-12-22\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2010-07-29\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2010-07-29\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/17972/2009-02-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2015-04-20\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2015-04-20\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2015-03-04\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2015-03-04\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2015-02-04\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2015-02-04\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2017-05-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2017-05-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2017-04-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/44638/2017-04-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/28059/2012-07-04\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/28059/2011-07-27\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/28059/2011-07-27\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/28059/2011-07-11\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/28059/2011-07-11\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/20817/2012-10-10\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/20817/2012-10-10\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/20817/2009-08-20\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/20817/2009-08-20\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/28973/2011-10-17\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-10-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-10-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-09-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-09-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-08-13\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-08-13\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-07-14\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-07-14\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-06-16\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/35169/2014-06-16\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

...

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54847/2017-01-19\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54847/2017-01-19\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54847/2016-12-15\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54847/2016-12-15\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54925/2016-12-20\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54925/2016-12-20\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54984/2016-12-22\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54984/2016-12-22\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54988/2016-12-22\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/54988/2016-12-22\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56074/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56074/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56075/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56075/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56076/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56076/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56077/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56077/2017-03-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56083/2017-03-07\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56083/2017-03-07\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56802/2017-04-11\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56802/2017-04-11\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56804/2017-04-11\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56804/2017-04-11\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56815/2017-04-18\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56815/2017-04-18\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56815/2017-05-19\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56815/2017-05-19\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56815/2017-05-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/56815/2017-05-05\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57518/2017-05-24\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57535/2017-05-24\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57535/2017-05-24\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57625/2017-05-31\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57625/2017-05-31\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57626/2017-05-31\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57626/2017-05-31\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57711/2017-06-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57711/2017-06-06\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/57776/2017-06-08\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymi

Number of corrupt files are157
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58778/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58778/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58794/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58794/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58802/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58802/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58805/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58805/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58806/2017-08-01\anom_explore_corrected_url.xml
N:/AUG-ImageRepository/anonymized_xml/loc_img_db/58806/2017-08-01\anom_explore_corrected_url.xml